1. importing the required libarires
2. Normalization of input data. purpose: Neural networks train faster and converge better when input values are centered (mean ≈ 0) and scaled.Helps with gradient descent stability.
3. loading train data from pytorch datasets train and test (data augumentation (not needed here))
4. load to dataloader, batches, shuffles, processes
5. Define classes
6. build neural network
7. define loss function
8. run epochs

In [1]:
#pip install torch torchvision torchsummary

In [2]:
# importing the required libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim #implementing optimization algorithms
from torchsummary import summary

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                               ])
                                                                    

In [4]:
# root is where data will be downloaded. "data" is the folder, if not will be created. train True means load the training data. applies transofrmation as mentioned in step2
trainset = torchvision.datasets.CIFAR10(root="./data", train=True,download=True,transform=transform) 

In [5]:
# batch size 4 indicates it processes 4 images at once, shuffle True means it shuffles at every epoch to avoid order based learning, num workers are subprocesses, you can use 0 also
trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=0)

In [6]:
testset=torchvision.datasets.CIFAR10(root="./data",train=False,download=True,transform=transform)

In [7]:
testloader= torch.utils.data.DataLoader(testset,batch_size=4,shuffle=False,num_workers=0)

In [8]:
#define classes
classes=('plane','car','bird','cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [9]:
class Net(nn.Module): #inherits nn.Module attributes
    def __init__(self):
        super(Net,self).__init__() # calls parent class initialization 
        self.conv1=nn.Conv2d(3,6,5) # 3 input channels , out channels, kenel size, strides, padding, dilation
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(torch.relu(self.conv1(x)))
        x=self.pool(torch.relu(self.conv2(x)))
        x=x.view(-1,16*5*5)
        x=torch.relu(self.fc1(x))
        x=torch.relu(self.fc2(x))
        x=self.fc3(x)
        return x


        
        
    

In [10]:
net=Net()

In [11]:
summary(net,input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


In [12]:
#loss function
criterion= nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [13]:
for epoch in range(2):
    running_loss=0.0
    for i,data in enumerate(trainloader,0):
        inputs,labels=data
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss +=loss.item()
        if i%2000 ==1999:
            print ('[%d,%5d] loss: %.3f' %
                  (epoch+1,i+1,running_loss/2000))
            running_loss=0.0
print("finished training")

[1, 2000] loss: 2.186
[1, 4000] loss: 1.852
[1, 6000] loss: 1.664
[1, 8000] loss: 1.581
[1,10000] loss: 1.532
[1,12000] loss: 1.468
[2, 2000] loss: 1.408
[2, 4000] loss: 1.382
[2, 6000] loss: 1.390
[2, 8000] loss: 1.357
[2,10000] loss: 1.319
[2,12000] loss: 1.276
finished training


In [14]:
data[1].shape

torch.Size([4])

In [15]:
correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs=net(images)
        _,predicted= torch.max(outputs.data,1)
        total +=labels.size(0)
        correct +=(predicted ==labels).sum().item()
print("accuracy of the network on the 10000 test images: %d %%" %(100*correct/total))        

accuracy of the network on the 10000 test images: 54 %


In [16]:
#running 4 epochs
for epoch in range(4):
    running_loss=0.0
    for i,data in enumerate(trainloader,0):
        inputs,labels=data
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss +=loss.item()
        if i%2000 ==1999:
            print ('[%d,%5d] loss: %.3f' %
                  (epoch+1,i+1,running_loss/2000))
            running_loss=0.0
print("finished training")

[1, 2000] loss: 1.212
[1, 4000] loss: 1.236
[1, 6000] loss: 1.208
[1, 8000] loss: 1.188
[1,10000] loss: 1.208
[1,12000] loss: 1.162
[2, 2000] loss: 1.119
[2, 4000] loss: 1.113
[2, 6000] loss: 1.113
[2, 8000] loss: 1.126
[2,10000] loss: 1.086
[2,12000] loss: 1.108
[3, 2000] loss: 1.020
[3, 4000] loss: 1.025
[3, 6000] loss: 1.041
[3, 8000] loss: 1.045
[3,10000] loss: 1.063
[3,12000] loss: 1.043
[4, 2000] loss: 0.948
[4, 4000] loss: 0.994
[4, 6000] loss: 0.971
[4, 8000] loss: 0.989
[4,10000] loss: 0.991
[4,12000] loss: 1.008
finished training


In [17]:
correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs=net(images)
        _,predicted= torch.max(outputs.data,1)
        total +=labels.size(0)
        correct +=(predicted ==labels).sum().item()
print("accuracy of the network on the 10000 test images: %d %%" %(100*correct/total))        

accuracy of the network on the 10000 test images: 59 %


The last two steps can be repeated with varying numbers of epochs and batch sizes (depending on memory), allowing for experimentation and fine-tuning.